## Imports

In [4]:
import numpy as np
import control as ct
import scipy as sp
import sympy as symp

## Hespanha 16.2

In [5]:
A = np.array([
    [1, 0, 0],
    [1, 1, 0],
    [-2, 1, 1]
])
B = np.array([
    [2],
    [0],
    [0]
])
C = np.array([
    [1, 0, 1]
])

k_1, k_2, k_3, s = symp.symbols('k_1 k_2 k_3 s')

k = symp.Matrix([[k_1], [k_2], [k_3]])

omega = ct.obsv(A, C)
print("Omega:")
print(np.linalg.matrix_rank(omega))

res = A + k@C
symp.pprint(res)

det = symp.det(s*symp.eye(3) - res)

print("Determinant:")
symp.pprint(det)
print(symp.latex(det))
poly = symp.Poly(det, s)

poly_target = symp.Poly((s+1)*(s+1)*(s+1), s)

print("Target polynomial Coefficients:")
symp.pprint(poly_target.all_coeffs())

print("Coefficients:")
symp.pprint(poly.all_coeffs())

print("Solve:")
solve_matrix = symp.Matrix([first - second for first, second in zip(poly.all_coeffs(), poly_target.all_coeffs())])
symp.pprint(solve_matrix)
sol = symp.solve(solve_matrix, [k_1, k_2, k_3], dict=True)[0]

print("Solution:")
symp.pprint(sol)

k = symp.Matrix([[sol[k_1]], [sol[k_2]], [sol[k_3]]])
symp.pprint(k)

k = np.array(k).astype(np.float64)

print("Eigs")
eigs = np.linalg.eigvals(A + k@C)
print(eigs)

L = ct.place(A.T, C.T, [-1, -2, -3]).T

print("L:")
symp.pprint(L)



Omega:
3
⎡k₁ + 1  0    k₁  ⎤
⎢                 ⎥
⎢k₂ + 1  1    k₂  ⎥
⎢                 ⎥
⎣k₃ - 2  1  k₃ + 1⎦
Determinant:
      2                                   2                  3      2         
- k₁⋅s  + 4⋅k₁⋅s - 4⋅k₁ - k₂⋅s + k₂ - k₃⋅s  + 2⋅k₃⋅s - k₃ + s  - 3⋅s  + 3⋅s - 

 
1
- k_{1} s^{2} + 4 k_{1} s - 4 k_{1} - k_{2} s + k_{2} - k_{3} s^{2} + 2 k_{3} s - k_{3} + s^{3} - 3 s^{2} + 3 s - 1
Target polynomial Coefficients:
[1, 3, 3, 1]
Coefficients:
[1, -k₁ - k₃ - 3, 4⋅k₁ - k₂ + 2⋅k₃ + 3, -4⋅k₁ + k₂ - k₃ - 1]
Solve:
⎡         0         ⎤
⎢                   ⎥
⎢   -k₁ - k₃ - 6    ⎥
⎢                   ⎥
⎢ 4⋅k₁ - k₂ + 2⋅k₃  ⎥
⎢                   ⎥
⎣-4⋅k₁ + k₂ - k₃ - 2⎦
Solution:
{k₁: -8, k₂: -28, k₃: 2}
⎡-8 ⎤
⎢   ⎥
⎢-28⎥
⎢   ⎥
⎣ 2 ⎦
Eigs
[-0.99998073+3.33692338e-05j -0.99998073-3.33692338e-05j
 -1.00003853+0.00000000e+00j]
L:
[[ 24.] 
 [ 74.] 
 [-15.]]


## Defining system matrices

In [6]:
A = np.array([
    [-5, 0, 0],
    [0, 3, 0],
    [0, 2, 1]
])

B = np.array([
    [0, 0],
    [0, 1],
    [1, 0]
])

C = np.array([
    [0, 0, 1],
    [-1, 0, 0]
])

## Checking the controllability and observability

In [7]:
gamma = ct.ctrb(A, B)
print(f'Rank of Gamma: {np.linalg.matrix_rank(gamma)}')

for eig in np.linalg.eigvals(A):
    print(f"Rank of (A - {eig} * I):", np.linalg.matrix_rank(np.concatenate((A - eig * np.eye(3), B), axis=1)))

omega = ct.obsv(A, C)
print(f'Rank of Omega: {np.linalg.matrix_rank(omega)}')

Rank of Gamma: 2
Rank of (A - -5.0 * I): 2
Rank of (A - 1.0 * I): 3
Rank of (A - 3.0 * I): 3
Rank of Omega: 3


## Designing the Feedback Matrix

In [8]:
# Constructing the controllablity decomposition transformation matrix
T = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
])
print('T:')
print(T)

print('Controllable dimension:', 2)

A_bar = np.linalg.inv(T) @ A @ T
B_bar = np.linalg.inv(T) @ B

print("A bar:")
print(A_bar)
print("B bar:")
print(B_bar)


A_bar_11 = A_bar[:2, :2]
B_bar_1 = B_bar[:2, :2]

print("A bar 11:")
print(A_bar_11)
print("B bar 1:")
print(B_bar_1)

x_hat_max = np.linalg.inv(T) @ np.array([[5], [3], [1]])
print("x_hat_max:")
print(x_hat_max)

x_hat_max = (1/(x_hat_max[:2])**2).flatten()

Q = np.diag(x_hat_max)
print("Q:")
print(Q)
R = np.diag([1/(10**1), 1])

K_bar = ct.lqr(A_bar_11, B_bar_1, Q, R)[0]
print("K bar:")
print(K_bar)
K = np.concatenate((K_bar, np.zeros((2,1))), axis=1) @ np.linalg.inv(T)
print("K using decomp:")
print(K)

print('Eigenvalues of A - BK:', np.linalg.eigvals(A - B @ K))



T:
[[0 0 1]
 [0 1 0]
 [1 0 0]]
Controllable dimension: 2
A bar:
[[ 1.  2.  0.]
 [ 0.  3.  0.]
 [ 0.  0. -5.]]
B bar:
[[1. 0.]
 [0. 1.]
 [0. 0.]]
A bar 11:
[[1. 2.]
 [0. 3.]]
B bar 1:
[[1. 0.]
 [0. 1.]]
x_hat_max:
[[1.]
 [3.]
 [5.]]
Q:
[[1.         0.        ]
 [0.         0.11111111]]
K bar:
[[4.28913879 1.34746447]
 [0.13474645 6.0770978 ]]
K using decomp:
[[0.         1.34746447 4.28913879]
 [0.         6.0770978  0.13474645]]
Eigenvalues of A - BK: [-3.18311829+0.27692327j -3.18311829-0.27692327j -5.        +0.j        ]


## Finding Luenberger Matrix

In [11]:
Q = np.diag([1/5**2, 1/3**2, 1])
R = np.diag([1/100, 1])
L = np.transpose(ct.lqr(A.T, C.T, Q, R)[0])

print('Eigenvalues of A - LC:', np.linalg.eigvals(A - L @ C))
print('L Matrix:')
print(L)

Eigenvalues of A - LC: [ -5.0039984  -10.02568424  -3.07987914]
L Matrix:
[[-3.10487500e-17 -3.99840128e-03]
 [ 3.95972929e+01  2.98193676e-21]
 [ 1.71055634e+01  3.10487500e-19]]
